<h1>Measuring Pi</h1>
<p>
A fun application of the Monte Carlo method is to approximate $\pi$ by simulating darts.  Take a look at <a href="https://github.com/drJAGartner/pi_day/blob/master/numberLine.png">this image</a> of a circle which is contained within a square.  Consider only the top $\frac{1}{4}$ of the circle/square.  If the circle radius is 1, we know that:</p>
$A_{circle} = \frac{\pi*r^2}{4} = \frac{\pi}{4}$
<p> and </p>
$A_{square} = (r)^2 = 1$

<br>
<p>It then follows that</p>
$\pi = 4*\frac{A_{circle}}{A_{square}} = 4 * R_{cs}$
<br>
<p>Where $R_{cs}$ is the ratio of the two areas.  One way of approximating this is by calculating the ratio of points inside the circle radius, to those in the square.  If we remember correctly, since points have no space, this is an infinite probability space!  Here we can use  the Monte Carlo method to simulate 'throwing darts', i.e. generating points in the square [0,1), [0,1) to approximate this Ratio.<p>


<p>Let's start by defining the functions we'll need</p>


In [3]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from math import sqrt
import random
import numpy as np
from scipy.optimize import curve_fit

def point():
#    return #return a tuple with an x and y value
    return (random.random(), random.random())

def in_circle(point):
#    return #return 1 if the point is in the circle, 0 otherwise
    return 1 if (point[0]*point[0])+(point[1]*point[1]) <= 1 else 0

def ratio(n_trials):
#    #your code here
    return (4.*np.sum([in_circle(point()) for x in range(n_trials)]))/n_trials

<h2>Run the trials</h2>
<p>Now that you have the apperatus to run an experement of n trials in length, you can just create a loop to run many experiments, and compare the accuracy of a particular experiment over varying length</p>

In [4]:
r3,r4,r5 = [],[],[]

#This will take a few minutes to run
n_exp = 1000
for i in range(n_exp):
    if i%(n_exp/10)==0:
        print "Completed {}%".format((100.*i)/n_exp)
    r3.append(ratio(1000))
    r4.append(ratio(10000))
    r5.append(ratio(100000))
    

Completed 0.0%
Completed 10.0%
Completed 20.0%
Completed 30.0%
Completed 40.0%
Completed 50.0%
Completed 60.0%
Completed 70.0%
Completed 80.0%
Completed 90.0%


<h2>Create Histograms of results, fit the data</h2>
<p>I define these functions for you, but if you don't understand any part, let me know and we can discuss</p>

In [5]:
def normal(x, mu, sig, c):
    return c*(1/sqrt(2*np.pi*sig*sig))*np.exp(-(x-mu)*(x-mu)/(2*sig*sig))

def stat_calc(ratios, bins, centers, curve_X):
    # Create a histogram of the measured ratios
    hist, edges = np.histogram(ratios, bins=bins)
    # Fit data to Normal Distribution
    fit_pars, covariance = curve_fit(normal, centers[1:-1], hist[1:-1], p0=(3.14, 0.1, 100))
    # Create data to plot the fit
    curve = normal(curve_X, *fit_pars)
    return (hist, fit_pars, curve)



<h2>Present the data</h2>
<p>You now have the data to describe both the simulation, and the fit of the data.  Try to think of a way to best communicate the results</p>

In [6]:
def plot_objects(ratios, bins, centers, curve_X, str_trials, str_color):
    hist, fit_pars, curve = stat_calc(ratios, bins, centers, curve_X)
    plot_data = [
        go.Scatter(
            name="{} Trials".format(str_trials),
            x=centers,
            y=hist,
            mode = 'markers',
            marker=dict(
                color=str_color,
            )
        ),
        go.Scatter(
            name="{} Trial Fit".format(str_trials),
            x=curve_X,
            y=curve,
            line = dict(
                color = str_color,
                width = 2,
                dash = 'dot'
            )
        )
    ]
    return plot_data, fit_pars[0]

In [7]:
bins = np.linspace(3.04, 3.24, 50)
# Use bin centers for accurate fitting/plotting
width = (bins[1]-bins[0])/2
centers = map(lambda x: x + width, bins[:-1])
curve_X = np.linspace(3.04, 3.24, 1000)

p3, mu3 = plot_objects(r3, bins, centers, curve_X, "1,000", "black")
p4, mu4 = plot_objects(r4, bins, centers, curve_X, "10,000", "gray")
p5, mu5 = plot_objects(r5, bins, centers, curve_X, "100,000", "red")

data = []
data.extend(p3)
data.extend(p4)
data.extend(p5)

In [8]:
annotations = [
    go.Annotation(
        x=3.05,
        y=300,
        text='mu_1,000: {}'.format(mu3),
        showarrow=False
    ),
    go.Annotation(
        x=3.05,
        y=285,
        text='mu_10,000: {}'.format(mu4),
        showarrow=False
    ),
    go.Annotation(
        x=3.05,
        y=270,
        text='mu_100,000: {}'.format(mu5),
        showarrow=False
    )
]


layout = go.Layout(
    title = 'Simulating pi via MC with N trials',
    xaxis = dict(
        title='Roll Result',
        titlefont = dict(
            size =14,
            color ='black'
        )
    ),
    yaxis=dict(
        title='N_occ',
        titlefont=dict(
            size=14,
            color='black'
        )
    ),
    shapes = [
        dict(
            type='line',
            x0=3.141592,
            y0= 0,
            x1= 3.141592,
            y1= 300,
            line=dict(
                color= 'green',
                width= 3,
            ),
        )
    ],
    annotations=annotations
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='dice_pmf_w_sim')